# 0- Imports

In [137]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
licenca ="default"

In [3]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

ampl = tools.ampl_notebook(
    modules=["highs", "gokestrel", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [4]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [5]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [6]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

Remover os espaços no nome das variáveis

In [7]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [8]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

## Caixa padrão

In [9]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [10]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [11]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [115]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [111]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [120]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [102]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [103]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [12]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [144]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [145]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,27,1,1,inf,1865_6.0,26_CX_5,3092300,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,21,1,1,inf,1865_6.0,26_CX_5,3321500,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,4,1,1,inf,1865_6.0,26_CX_5,3537200,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,179,1,3,inf,2489_674.0,116_CX_6,233624,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,109,1,1,inf,2988_451.0,16_CX_2,775500,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,27778950.0,83393894.4,75538008.0


In [146]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [147]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [148]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,27778950.0,83393894.4,75538008.0,75538008.0,512


In [149]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

# 2- Modelagem

## 2.1- CSP com problemas do negócio - múltiplas chaves

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i k}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na mochila $j$, para a loja $k$.


In [ ]:
%%ampl_eval
reset;

param n;
param cv;
param cp;
param np;
param r;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
set K = 1..r ordered;
param w {I,K} >= 0;
param v {I,K} >= 0;
param inner{I,K} >= 0;
param d {I,K} >= 0;

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
#param maxVal_volume := max {i in I, k in K} v[i,k];
#param pecas_por_caixa_peso := floor(cp / maxVal_peso);
#param pecas_por_caixa_volume := floor(cv / maxVal_volume);
#param pecas_caixa := min(pecas_por_caixa_peso, pecas_por_caixa_volume);
param pecas_caixa:= floor(cp / maxVal_peso);


param soma_coluna {k in K} := sum{i in I} d[i,k];
#param maior_soma := max{k in K} soma_coluna[k];
param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0;
var y {J,K} binary;

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.1.1- Importar dados

In [ ]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [ ]:
df_process.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")').groupby(['chave_loja_buffer_onda']).agg({'Peso_unit':'sum', 'Volume_unit':'sum', 'Peças':'sum'})\
                                                                                                                .sort_values(by='Peças', ascending=False)

,Peso_unit,Volume_unit,Peças
chave_loja_buffer_onda,,,
16_EN_3,80.349,1208882177,2093
33_EN_3,94.018,1430210694,1786
9_EN_4,134.444,1964692726,1464


In [ ]:
df_process_situacao1 = df_process.query('chave_loja_buffer_onda in ["16_EN_3","9_EN_4","33_EN_3"]')

df_process_situacao1

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit
33868,33868,40,2023-10-30,847254655,443,233,32,0.303,3,CARTON_M,...,EN,40,28.0,21,1,1,inf,40_28.0,9_EN_4,3303008
33869,33869,915,2023-10-30,847254621,445,242,33,0.323,3,CARTON_M,...,EN,40,28.0,31,1,1,inf,40_28.0,9_EN_4,3553770
33870,33870,4051,2023-10-30,847254663,431,225,34,0.295,2,CARTON_M,...,EN,40,28.0,4,1,1,inf,40_28.0,9_EN_4,3297150
33871,33871,8788,2023-10-30,847254680,445,244,33,0.328,2,CARTON_M,...,EN,40,28.0,8,1,1,inf,40_28.0,9_EN_4,3583140
33872,33872,8438,2023-10-30,847254639,432,248,32,0.310,4,CARTON_M,...,EN,40,28.0,39,1,1,inf,40_28.0,9_EN_4,3428352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59786,59786,1204,2023-11-21,879747286,321,146,45,0.240,5,CARTON_M,...,EN,974,343.0,1,1,1,inf,974_343.0,9_EN_4,2108970
59787,59787,4234,2023-11-21,879747315,309,156,65,0.203,2,CARTON_M,...,EN,974,343.0,13,1,1,inf,974_343.0,9_EN_4,3133260
59788,59788,8697,2023-11-21,879747294,314,134,41,0.230,7,CARTON_M,...,EN,974,343.0,5,1,1,inf,974_343.0,9_EN_4,1725116
59789,59789,9768,2023-11-21,879747307,320,147,42,0.217,3,CARTON_M,...,EN,974,343.0,3,1,1,inf,974_343.0,9_EN_4,1975680


### 2.1.2- Entrada de dados no modelo

Matriz peso unitário

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,0.220,0.207,0.303
1,2,0.167,0.213,0.323
2,3,0.240,0.156,0.295
3,4,0.213,0.150,0.328
4,5,0.185,0.455,0.310
...,...,...,...,...
405,406,0.000,0.000,0.240
406,407,0.000,0.000,0.203
407,408,0.000,0.000,0.230
408,409,0.000,0.000,0.217


Matriz volume unitário

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,3001320.0,3228148.0,3303008.0
1,2,2741220.0,2456160.0,3553770.0
2,3,2864650.0,1212300.0,3297150.0
3,4,2456160.0,2290431.0,3583140.0
4,5,2637180.0,4548648.0,3428352.0
...,...,...,...,...
405,406,0.0,0.0,2108970.0
406,407,0.0,0.0,3133260.0
407,408,0.0,0.0,1725116.0
408,409,0.0,0.0,1975680.0


Matriz inner

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
405,406,0.0,0.0,1.0
406,407,0.0,0.0,1.0
407,408,0.0,0.0,1.0
408,409,0.0,0.0,1.0


Matriz demanda

In [ ]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao1['item_loja'] = df_process_situacao1.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao1.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,3.0
1,2,1.0,1.0,3.0
2,3,4.0,1.0,2.0
3,4,2.0,2.0,2.0
4,5,1.0,1.0,4.0
...,...,...,...,...
405,406,0.0,0.0,5.0
406,407,0.0,0.0,2.0
407,408,0.0,0.0,7.0
408,409,0.0,0.0,3.0


In [ ]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

In [ ]:
buffer = df_process_situacao1['Tipo_de_buffer'].unique()[0]
cp = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peso_max_(kg)'].values
cv = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['volume_util_caixa'].values
np = df_caixas_padrao.query(f'Tipo_de_buffer == "{buffer}"')['Peças_max'].values

print(cp, cv, np)

[23] [75538008.] [512]


#### Parâmetros

In [ ]:
ampl.param["n"] = len(pecas)   # tamanho da loa que tem mais itens

ampl.param["cp"] = cp
ampl.param["cv"] = cv
ampl.param["np"] = np
ampl.param["r"] = len(volume.columns) -1

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

In [ ]:
ampl.getValue('soma')

5343

In [ ]:
ampl.getValue('maxbins')

268

In [ ]:
ampl.getValue('pecas_caixa')

20

### 2.1.3- Solvers

#### Solver GCG

In [ ]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1 timelimit=200"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
  lim:time = 200
 added complete decomp for original problem with 268 blocks and 1041 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.695917 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 267 upgd conss, 0 impls, 267 clqs
   (2.0s) probing: 51/72360 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (2.0s) probing aborted: 50/50 successive totally useless probings
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 267 cliques
presolved problem has 279792 variables (72360 bin, 207432 int, 0 impl, 0 cont) and 3453 constraints
    267 constraints of type <setppc>
   3186 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 2.00
 calculated translatio

In [ ]:
%%ampl_eval
display {j in J, k in K} y[j,k];

y[j,k] [*,*]
:     1   2   3    :=
1     0   0   0
2     0   0   0
3     0   0   0
4     0   0   0
5     0   0   0
6     0   0   0
7     0   0   0
8     0   0   0
9     0   0   0
10    0   0   0
11    0   0   0
12    0   0   0
13    0   0   0
14    0   0   0
15    0   0   0
16    0   0   0
17    0   0   0
18    0   0   0
19    0   0   0
20    0   0   0
21    0   0   0
22    0   0   0
23    0   0   0
24    0   0   0
25    0   0   0
26    0   0   0
27    0   0   0
28    0   0   0
29    0   0   0
30    0   0   0
31    0   0   0
32    0   0   0
33    0   0   0
34    0   0   0
35    0   0   0
36    0   0   0
37    0   0   0
38    0   0   0
39    0   0   0
40    0   0   0
41    0   0   0
42    0   0   0
43    0   0   0
44    0   0   0
45    0   0   0
46    0   0   0
47    0   0   0
48    0   0   0
49    0   0   0
50    0   0   0
51    0   0   0
52    0   0   0
53    0   0   0
54    0   0   0
55    0   0   0
56    0   0   0
57    0   0   0
58    0   0   0
59    0   0   0
60    0   0   0
61   

In [ ]:
e_s1 = ampl.getVariable('e').getValues().toPandas()
e_s1 = e_s1.reset_index()
e_s1.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1 = e_s1.query('valor_multiplo >0')
e_s1#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo


In [ ]:
e_s1.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)

caixa
item_loja


In [ ]:
df_agrupamento_loja1 = e_s1.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1 = df_agrupamento_loja1.reset_index()
df_agrupamento_loja1.columns.name = None
df_agrupamento_loja1 # as colunas são as caixas

,item_loja


In [ ]:
df_agrupamento_loja2 = e_s1.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2 = df_agrupamento_loja2.reset_index()
df_agrupamento_loja2.columns.name = None
df_agrupamento_loja2 # as colunas são as caixas

,item_loja


In [ ]:
df_agrupamento_loja3 = e_s1.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3 = df_agrupamento_loja3.reset_index()
df_agrupamento_loja3.columns.name = None
df_agrupamento_loja3 # as colunas são as caixas

,item_loja


Identificação do id de cada item

In [ ]:
#Pegar a ordem das lojas:
w.columns

Index(['16_EN_3', '33_EN_3', '9_EN_4'], dtype='object')

In [ ]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao1.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [ ]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [ ]:
for i in range(0,3):
    print(f'tamanho do vetor {i+1} (chave "{w.columns[i]}"): {len(vetores[f"{w.columns[i]}"])}')

tamanho do vetor 1 (chave "16_EN_3"): 311
tamanho do vetor 2 (chave "33_EN_3"): 320
tamanho do vetor 3 (chave "9_EN_4"): 410


In [ ]:
df_agrupamento_loja1['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1

,item_loja,Id,chave_loja_buffer_onda
0,NaN,34013,16_EN_3
1,NaN,34014,16_EN_3
2,NaN,34015,16_EN_3
3,NaN,34016,16_EN_3
4,NaN,34017,16_EN_3
...,...,...,...
306,NaN,57776,16_EN_3
307,NaN,57777,16_EN_3
308,NaN,57778,16_EN_3
309,NaN,57779,16_EN_3


In [ ]:
df_agrupamento_loja2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2

,item_loja,Id,chave_loja_buffer_onda
0,NaN,34201,33_EN_3
1,NaN,34202,33_EN_3
2,NaN,34203,33_EN_3
3,NaN,34204,33_EN_3
4,NaN,37205,33_EN_3
...,...,...,...
315,NaN,58412,33_EN_3
316,NaN,58413,33_EN_3
317,NaN,58414,33_EN_3
318,NaN,58415,33_EN_3


In [ ]:
df_agrupamento_loja3['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3

,item_loja,Id,chave_loja_buffer_onda
0,NaN,33868,9_EN_4
1,NaN,33869,9_EN_4
2,NaN,33870,9_EN_4
3,NaN,33871,9_EN_4
4,NaN,33872,9_EN_4
...,...,...,...
405,NaN,59786,9_EN_4
406,NaN,59787,9_EN_4
407,NaN,59788,9_EN_4
408,NaN,59789,9_EN_4


Caixas selecionadas para cada loja:

In [ ]:
y_s1 = ampl.getVariable('y').getValues().toPandas()
y_s1 = y_s1.reset_index()
y_s1.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1 = y_s1.query('selecao>0')

df_caixas_por_loja = y_s1.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja = df_caixas_por_loja.reset_index()
df_caixas_por_loja.columns.name = None
df_caixas_por_loja # as colunas são as caixas

,Caixa


In [ ]:
df_caixas_por_loja.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja

ValueError: Length mismatch: Expected axis has 1 elements, new values have 4 elements

In [ ]:
print('Solver GCG')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja[f"{col}"].sum()}')

#### Solver Highs

In [ ]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [8e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+01]
  RHS    [1e+00, 3e+01]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      0            0            0
Row     residuals            0            0            0
Presolving model
3453 rows, 279792 cols, 1118364 nonzeros  0s
3453 rows, 279792 cols, 1118364 nonzeros  5s

MIP start solution is feasible, objective value is 361
Objective function is integral with scale 1

Solving MIP model with:
   3453 rows
   279792 cols (72360 binary, 207432 integer, 0 implied int., 0 continuous)
   1118364 nonzeros

        Nodes      |    

In [ ]:
e_s1_highs = ampl.getVariable('e').getValues().toPandas()
e_s1_highs = e_s1_highs.reset_index()
e_s1_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_highs = e_s1_highs.query('valor_multiplo >0')
e_s1_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
53,1,18,3,2.0
271,1,91,2,1.0
507,1,170,1,1.0
764,1,255,3,1.0
1196,2,131,3,3.0
...,...,...,...,...
326999,407,192,3,1.0
327833,408,202,3,1.0
327947,408,240,3,6.0
328112,409,27,3,3.0


In [ ]:
df_agrupamento_loja1_highs = e_s1_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs = df_agrupamento_loja1_highs.reset_index()
df_agrupamento_loja1_highs.columns.name = None
df_agrupamento_loja1_highs # as colunas são as caixas

,item_loja,1,2,13,18,20,22,23,24,27,...,243,244,248,257,259,260,261,262,266,267
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_agrupamento_loja1_highs['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs

,item_loja,1,2,13,18,20,22,23,24,27,...,248,257,259,260,261,262,266,267,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34014,16_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,34015,16_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [ ]:
df_agrupamento_loja2_highs = e_s1_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs = df_agrupamento_loja2_highs.reset_index()
df_agrupamento_loja2_highs.columns.name = None
df_agrupamento_loja2_highs # as colunas são as caixas

,item_loja,1,7,9,10,11,12,13,14,17,...,232,233,240,245,246,254,256,259,263,265
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_agrupamento_loja2_highs['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs

,item_loja,1,7,9,10,11,12,13,14,17,...,240,245,246,254,256,259,263,265,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34201,33_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34202,33_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34203,33_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34204,33_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37205,33_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58412,33_EN_3
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58413,33_EN_3
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58414,33_EN_3
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58415,33_EN_3


In [ ]:
df_agrupamento_loja3_highs = e_s1_highs.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_highs = df_agrupamento_loja3_highs.reset_index()
df_agrupamento_loja3_highs.columns.name = None
df_agrupamento_loja3_highs # as colunas são as caixas

,item_loja,8,17,18,23,26,27,31,38,39,...,249,250,253,255,256,259,260,265,267,268
0,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,406,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
406,407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
408,409,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_agrupamento_loja3_highs['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_highs['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_highs

,item_loja,8,17,18,23,26,27,31,38,39,...,253,255,256,259,260,265,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,33868,9_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33869,9_EN_4
2,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33870,9_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33871,9_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,33872,9_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,406,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59786,9_EN_4
406,407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59787,9_EN_4
407,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59788,9_EN_4
408,409,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59789,9_EN_4


In [ ]:
df_process_situacao1.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'item_loja'],
      dtype='object')

In [ ]:
df_agrupamento_loja3_highs.merge(df_process_situacao1[['Id', 'Peças', 'Peso_unit','Volume_unit', 'Inner']], on='Id', how= 'inner')

,item_loja,8,17,18,23,26,27,31,38,39,...,260,265,267,268,Id,chave_loja_buffer_onda,Peças,Peso_unit,Volume_unit,Inner
0,1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33868,9_EN_4,3,0.303,3303008,1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33869,9_EN_4,3,0.323,3553770,1
2,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33870,9_EN_4,2,0.295,3297150,1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33871,9_EN_4,2,0.328,3583140,1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,33872,9_EN_4,4,0.310,3428352,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,406,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59786,9_EN_4,5,0.240,2108970,1
406,407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59787,9_EN_4,2,0.203,3133260,1
407,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59788,9_EN_4,7,0.230,1725116,1
408,409,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,59789,9_EN_4,3,0.217,1975680,1


In [ ]:
y_s1_highs = ampl.getVariable('y').getValues().toPandas()
y_s1_highs = y_s1_highs.reset_index()
y_s1_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_highs = y_s1_highs.query('selecao>0')

df_caixas_por_loja_highs = y_s1_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs = df_caixas_por_loja_highs.reset_index()
df_caixas_por_loja_highs.columns.name = None
df_caixas_por_loja_highs # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,0.0
1,2,1.0,0.0,0.0
2,7,0.0,1.0,0.0
3,8,0.0,0.0,1.0
4,9,0.0,1.0,0.0
...,...,...,...,...
201,263,0.0,1.0,0.0
202,265,0.0,1.0,1.0
203,266,1.0,0.0,0.0
204,267,1.0,0.0,1.0


In [ ]:
df_caixas_por_loja_highs.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs

,caixas,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,0.0
1,2,1.0,0.0,0.0
2,7,0.0,1.0,0.0
3,8,0.0,0.0,1.0
4,9,0.0,1.0,0.0
...,...,...,...,...
201,263,0.0,1.0,0.0
202,265,0.0,1.0,1.0
203,266,1.0,0.0,0.0
204,267,1.0,0.0,1.0


In [ ]:
print('Solver Highs')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs[f"{col}"].sum()}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "16_EN_3": 116.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 117.0
número de caixas para a chave_loja_buffer_onda "9_EN_4": 94.0


#### Solver SCIP

In [ ]:
ampl.option["solver"] = "scip"
ampl.option["scip_options"] = "outlev=1 timelimit=600"
ampl.solve()

SCIP 9.0.0:   tech:outlev = 1
  lim:time = 600
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 267 upgd conss, 0 impls, 267 clqs
   (3.0s) probing: 51/72360 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (3.0s) probing aborted: 50/50 successive totally useless probings
   (3.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (7.0s) symmetry computation finished: 228 generators found (max: 228, log10 of symmetry group size: inf) (symcode time: 3.00)
dynamic symmetry handling statistics:
   orbitopal reduction:        1 components: 268x3
   orbital reduction:          1 components of sizes 208
   lexicographic reduction:  226 permutations with support sizes 624, 624, 536, 536, 624, 536, 536, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 536, 536, 536, 536, 624, 536, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 62

In [ ]:
%%ampl_eval
display {j in J, k in K} y[j,k];

y[j,k] [*,*]
:     1   2   3    :=
1     1   1   1
2     1   1   1
3     1   1   1
4     1   1   1
5     1   1   1
6     1   1   1
7     1   1   1
8     1   1   1
9     1   1   1
10    1   1   1
11    1   1   1
12    1   1   1
13    1   1   1
14    1   1   1
15    1   1   1
16    1   1   1
17    1   1   1
18    1   1   1
19    1   1   1
20    1   1   1
21    1   1   1
22    1   1   1
23    1   1   1
24    1   1   1
25    1   1   1
26    1   1   0
27    1   1   0
28    1   1   1
29    1   1   0
30    1   1   1
31    1   1   0
32    1   1   1
33    1   1   0
34    1   1   1
35    1   1   0
36    1   1   1
37    1   1   1
38    1   1   1
39    1   1   1
40    1   1   0
41    1   1   1
42    1   1   1
43    1   1   0
44    1   1   1
45    1   1   0
46    1   1   0
47    1   1   1
48    1   1   0
49    1   1   1
50    1   1   0
51    1   1   1
52    1   1   0
53    1   1   0
54    1   1   0
55    1   1   0
56    1   1   0
57    1   1   1
58    1   1   1
59    1   1   1
60    1   1   1
61   

In [ ]:
e_s1_scip = ampl.getVariable('e').getValues().toPandas()
e_s1_scip = e_s1_scip.reset_index()
e_s1_scip.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_scip = e_s1_scip.query('valor_multiplo >0')
e_s1_scip#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
38,1,13,3,3
799,1,267,2,1
801,1,268,1,1
845,2,14,3,3
1605,2,268,1,1
...,...,...,...,...
325979,406,120,3,5
326669,407,82,3,2
327614,408,129,3,7
328115,409,28,3,3


In [ ]:
df_agrupamento_loja1_scip = e_s1_scip.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_scip = df_agrupamento_loja1_scip.reset_index()
df_agrupamento_loja1_scip.columns.name = None
df_agrupamento_loja1_scip['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_scip['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_scip

,item_loja,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34014,16_EN_3
2,3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34015,16_EN_3
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [ ]:
df_agrupamento_loja2_scip = e_s1_scip.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_scip = df_agrupamento_loja2_scip.reset_index()
df_agrupamento_loja2_scip.columns.name = None
df_agrupamento_loja2_scip['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_scip['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_scip

,item_loja,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,34201,33_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34202,33_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34203,33_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34204,33_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37205,33_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58412,33_EN_3
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58413,33_EN_3
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58414,33_EN_3
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,58415,33_EN_3


In [ ]:
df_agrupamento_loja3_scip = e_s1_scip.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_scip = df_agrupamento_loja3_scip.reset_index()
df_agrupamento_loja3_scip.columns.name = None
df_agrupamento_loja3_scip['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_scip['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_scip

,item_loja,1,2,3,4,5,6,7,8,9,...,257,259,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33868,9_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33869,9_EN_4
2,3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33870,9_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,33871,9_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33872,9_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59786,9_EN_4
406,407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59787,9_EN_4
407,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59788,9_EN_4
408,409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59789,9_EN_4


In [ ]:
y_s1_scip = ampl.getVariable('y').getValues().toPandas()
y_s1_scip = y_s1_scip.reset_index()
y_s1_scip.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_scip = y_s1_scip.query('selecao>0')

df_caixas_por_loja_scip = y_s1_scip.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_scip = df_caixas_por_loja_scip.reset_index()
df_caixas_por_loja_scip.columns.name = None
df_caixas_por_loja_scip # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
168,264,1.0,1.0,1.0
169,265,1.0,1.0,1.0
170,266,1.0,1.0,1.0
171,267,1.0,1.0,1.0


In [ ]:
df_caixas_por_loja_scip.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_scip

,caixas,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
168,264,1.0,1.0,1.0
169,265,1.0,1.0,1.0
170,266,1.0,1.0,1.0
171,267,1.0,1.0,1.0


In [ ]:
print('Solver Highs')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_scip[f"{col}"].sum()}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "16_EN_3": 119.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 118.0
número de caixas para a chave_loja_buffer_onda "9_EN_4": 115.0


#### Solver CBC

In [ ]:
ampl.option["solver"] = "cbc"
ampl.option["cbc_options"] = "outlev=1 timelimit=600"
ampl.solve()

cbc 2.10.10:   tech:outlev = 1
  lim:time = 600
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 18 2023 

command line - Cbc_C_Interface -log 1 -solve -quit (default strategy 1)
Continuous objective value is 322.601 - 12.76 seconds
Cgl0004I processed model has 3453 rows, 279792 columns (279792 integer (72360 of which binary)) and 1118364 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I MIPStart provided solution with cost 352
Cbc0012I Integer solution of 352 found by Reduced search after 0 iterations and 0 nodes (14.90 seconds)
Cbc0038I Full problem 3453 rows 279792 columns, reduced to 1911 rows 2241 columns
Cbc0031I 550 added rows had average density of 542.46909
Cbc0013I At root node, 550 cuts changed objective from 322.60136 to 322.60136 in 10 passes
Cbc0014I Cut generator 0 (Probing) - 130 row cuts average 4.1 elements, 0 column cuts (464 active)  in 6.034 seconds - new frequency is -100
Cbc0014I Cut generator 1 (Gomory) - 2056 row cuts average 11

In [ ]:
e_s1_cbc = ampl.getVariable('e').getValues().toPandas()
e_s1_cbc = e_s1_cbc.reset_index()
e_s1_cbc.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s1_cbc = e_s1_cbc.query('valor_multiplo >0')
e_s1_cbc#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
38,1,13,3,3
799,1,267,2,1
801,1,268,1,1
845,2,14,3,3
1605,2,268,1,1
...,...,...,...,...
325979,406,120,3,5
326669,407,82,3,2
327614,408,129,3,7
328115,409,28,3,3


In [ ]:
df_agrupamento_loja1_cbc = e_s1_cbc.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_cbc = df_agrupamento_loja1_cbc.reset_index()
df_agrupamento_loja1_cbc.columns.name = None
df_agrupamento_loja1_cbc['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_cbc['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_cbc

,item_loja,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34014,16_EN_3
2,3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34015,16_EN_3
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [ ]:
df_agrupamento_loja2_cbc = e_s1_cbc.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_cbc = df_agrupamento_loja2_cbc.reset_index()
df_agrupamento_loja2_cbc.columns.name = None
df_agrupamento_loja2_cbc['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_cbc['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_cbc

,item_loja,1,2,3,4,5,6,7,8,9,...,261,262,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,34201,33_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34202,33_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34203,33_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34204,33_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37205,33_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58412,33_EN_3
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58413,33_EN_3
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58414,33_EN_3
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,58415,33_EN_3


In [ ]:
df_agrupamento_loja3_cbc = e_s1_cbc.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_cbc = df_agrupamento_loja3_cbc.reset_index()
df_agrupamento_loja3_cbc.columns.name = None
df_agrupamento_loja3_cbc['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_cbc['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_cbc

,item_loja,1,2,3,4,5,6,7,8,9,...,257,259,263,264,265,266,267,268,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33868,9_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33869,9_EN_4
2,3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33870,9_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,33871,9_EN_4
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33872,9_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59786,9_EN_4
406,407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59787,9_EN_4
407,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59788,9_EN_4
408,409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59789,9_EN_4


In [ ]:
y_s1_cbc = ampl.getVariable('y').getValues().toPandas()
y_s1_cbc = y_s1_cbc.reset_index()
y_s1_cbc.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s1_cbc = y_s1_cbc.query('selecao>0')

df_caixas_por_loja_cbc = y_s1_cbc.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_cbc = df_caixas_por_loja_cbc.reset_index()
df_caixas_por_loja_cbc.columns.name = None
df_caixas_por_loja_cbc.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_cbc

,caixas,16_EN_3,33_EN_3,9_EN_4
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
168,264,1.0,1.0,1.0
169,265,1.0,1.0,1.0
170,266,1.0,1.0,1.0
171,267,1.0,1.0,1.0


In [ ]:
print('Solver Highs')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_cbc[f"{col}"].sum()}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "16_EN_3": 119.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 118.0
número de caixas para a chave_loja_buffer_onda "9_EN_4": 115.0


## 2.2- CSP com problemas do negócio - múltiplos tipo de buffer

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i}$, um volume $v_{i}$, um múltiplo $inner_{i k}$ da quantiade de itens a serem selecioner uma demanda $d_{i}$, para todo $i \in \{1,...,n\}$, e um número $m$ de mochilas (caixas) idênticas com capacidade volumétrica $cv$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np$, então temos:

$$
\begin{aligned}
\text{minimizar} \quad &\sum_{j = 1}^{m} \sum_{k = 1}^{r} y_{j k} \\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} = d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\
&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na mochila $j$, para a loja $k$.


In [237]:
%%ampl_eval
reset;

param n;
param cp;
param r;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
set K = 1..r ordered;
param w {I,K} >= 0;
param v {I,K} >= 0;
param inner{I,K} >= 0;
param d {I,K} >= 0;

param cv {K} >= 0;
param np {K} >= 0;

param maxVal := max {i in I, k in K} w[i,k];

param maxVal_peso := max {i in I, k in K} w[i,k];
param pecas_caixa:= floor(cp / maxVal_peso);

param soma := sum{i in I, k in K} d[i, k];
param maxbins := ceil(soma / pecas_caixa);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0;
var y {J,K} binary;

minimize FO:  sum {j in J, k in K} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

### 2.2.1- Importar dados

Vamos alterar a umas das chaves para incluir um loja com tipo de buffer distinto (chave: *10_CX_1*)

In [238]:
df_process_situacao2 = df_process_capacitado.query('chave_loja_buffer_onda in ["16_EN_3","10_CX_1","33_EN_3"]')

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,8,4.000000e+10,777_1.0,10_CX_1,5470920,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,8,4.000000e+10,777_1.0,10_CX_1,5470920,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,8,4.000000e+10,777_95.0,10_CX_1,1750000,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,8,4.000000e+10,777_95.0,10_CX_1,1750000,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,8,4.000000e+10,777_95.0,10_CX_1,1375000,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,1,inf,460_137.0,33_EN_3,3971916,27778950.0,83393894.4,75538008.0,75538008.0,512
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,1,inf,460_137.0,33_EN_3,1384435,27778950.0,83393894.4,75538008.0,75538008.0,512
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,1,inf,460_137.0,33_EN_3,1647536,27778950.0,83393894.4,75538008.0,75538008.0,512
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,1,inf,460_471.0,33_EN_3,5453000,27778950.0,83393894.4,75538008.0,75538008.0,512


In [239]:
df_process_situacao2['Peças_max'].unique()

array([170, 512], dtype=int64)

In [240]:
df_process_situacao2.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'volume_util_caixa_Caixa_1',
       'volume_util_caixa_Caixa_2', 'volume_util_caixa_Caixa_3',
       'volume_util_padrao', 'Peças_max'],
      dtype='object')

### 2.2.2- Entrada de dados no modelo

In [241]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,0.090,0.220,0.207
1,2,0.090,0.167,0.213
2,3,0.200,0.240,0.156
3,4,0.200,0.213,0.150
4,5,0.180,0.185,0.455
...,...,...,...,...
399,400,0.027,0.000,0.000
400,401,0.147,0.000,0.000
401,402,0.137,0.000,0.000
402,403,0.143,0.000,0.000


In [242]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,5470920.0,3001320.0,3228148.0
1,2,5470920.0,2741220.0,2456160.0
2,3,1750000.0,2864650.0,1212300.0
3,4,1750000.0,2456160.0,2290431.0
4,5,1375000.0,2637180.0,4548648.0
...,...,...,...,...
399,400,670670.0,0.0,0.0
400,401,3555576.0,0.0,0.0
401,402,3482136.0,0.0,0.0
402,403,3961375.0,0.0,0.0


In [243]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
399,400,1.0,0.0,0.0
400,401,1.0,0.0,0.0
401,402,1.0,0.0,0.0
402,403,1.0,0.0,0.0


In [244]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,16_EN_3,33_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,2.0,4.0,1.0
3,4,2.0,2.0,2.0
4,5,1.0,1.0,1.0
...,...,...,...,...
399,400,4.0,0.0,0.0
400,401,9.0,0.0,0.0
401,402,6.0,0.0,0.0
402,403,9.0,0.0,0.0


In [245]:
df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max,item_loja
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,4.000000e+10,777_1.0,10_CX_1,5470920,27778950.0,83393894.4,75538008.0,83393894.4,170,1
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,4.000000e+10,777_1.0,10_CX_1,5470920,27778950.0,83393894.4,75538008.0,83393894.4,170,2
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,4.000000e+10,777_95.0,10_CX_1,1750000,27778950.0,83393894.4,75538008.0,83393894.4,170,3
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,4.000000e+10,777_95.0,10_CX_1,1750000,27778950.0,83393894.4,75538008.0,83393894.4,170,4
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,4.000000e+10,777_95.0,10_CX_1,1375000,27778950.0,83393894.4,75538008.0,83393894.4,170,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58412,58412,761,2023-11-21,869448768,702,246,23,0.147,12,CARTON_L,...,inf,460_137.0,33_EN_3,3971916,27778950.0,83393894.4,75538008.0,75538008.0,512,316
58413,58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,...,inf,460_137.0,33_EN_3,1384435,27778950.0,83393894.4,75538008.0,75538008.0,512,317
58414,58414,761,2023-11-21,869448805,407,253,16,0.080,4,CARTON_M,...,inf,460_137.0,33_EN_3,1647536,27778950.0,83393894.4,75538008.0,75538008.0,512,318
58415,58415,761,2023-11-21,848470202,410,380,35,0.200,1,CARTON_M,...,inf,460_471.0,33_EN_3,5453000,27778950.0,83393894.4,75538008.0,75538008.0,512,319


Capacidade da caixa como variável

In [246]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,16_EN_3,512.0
2,33_EN_3,512.0


In [247]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,16_EN_3,75538008.0
2,33_EN_3,75538008.0


Características das peças

In [248]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner

Características da caixa

In [249]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [265]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1

cv_ = cv.drop(columns='chave_loja_buffer_onda')
np_ = np.drop(columns='chave_loja_buffer_onda')

v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')

ampl.param["w"] = {(i, k): w.iloc[i - 1][k - 1] for i in range(1, len(w)+1) for k in range(1, len(w.columns)+1)}
ampl.param["v"] = {(i, k): v.iloc[i - 1][k - 1] for i in range(1, len(v)+1) for k in range(1, len(v.columns)+1)}
ampl.param["d"] = {(i, k): d.iloc[i - 1][k - 1] for i in range(1, len(d)+1) for k in range(1, len(d.columns)+1)}
ampl.param["inner"] = {(i, k): inner_.iloc[i - 1][k - 1] for i in range(1, len(inner_)+1) for k in range(1, len(inner_.columns)+1)}

ampl.param["cv"] = {(k): cv_.iloc[k - 1] for k in range(1, len(cv_) + 1)}
ampl.param["np"] = {(k): np_.iloc[k - 1] for k in range(1, len(np_)+1)}

In [266]:
ampl.getValue('soma')

5827

In [267]:
ampl.getValue('maxbins')

225

In [268]:
ampl.getValue('pecas_caixa')

26

### 2.2.3- Solvers

#### Solver GCG

In [269]:
ampl.option["solver"] = "gcg"
ampl.option["gcg_options"] = "outlev=1 timelimit=600"
ampl.solve()

GCG 4.0.0:   tech:outlev = 1
  lim:time = 600
 added complete decomp for original problem with 225 blocks and 1035 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.658824 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 299 upgd conss, 0 impls, 299 clqs
   (2.0s) probing: 51/67950 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (2.0s) probing aborted: 50/50 successive totally useless probings
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 299 cliques
presolved problem has 233550 variables (67950 bin, 165600 int, 0 impl, 0 cont) and 3060 constraints
    299 constraints of type <setppc>
   2761 constraints of type <linear>
transformed objective value is always integral (scale: 1)
Presolving Time: 3.00
 calculated translatio

In [270]:
%%ampl_eval
display {j in J, k in K} y[j,k];

y[j,k] [*,*]
:     1   2   3    :=
1     0   0   0
2     0   0   0
3     0   0   0
4     0   0   0
5     0   0   0
6     0   0   0
7     0   0   0
8     0   0   0
9     0   0   0
10    0   0   0
11    0   0   0
12    0   0   0
13    0   0   0
14    0   0   0
15    0   0   0
16    0   0   0
17    0   0   0
18    0   0   0
19    0   0   0
20    0   0   0
21    0   0   0
22    0   0   0
23    0   0   0
24    0   0   0
25    0   0   0
26    0   0   0
27    0   0   0
28    0   0   0
29    0   0   0
30    0   0   0
31    0   0   0
32    0   0   0
33    0   0   0
34    0   0   0
35    0   0   0
36    0   0   0
37    0   0   0
38    0   0   0
39    0   0   0
40    0   0   0
41    0   0   0
42    0   0   0
43    0   0   0
44    0   0   0
45    0   0   0
46    0   0   0
47    0   0   0
48    0   0   0
49    0   0   0
50    0   0   0
51    0   0   0
52    0   0   0
53    0   0   0
54    0   0   0
55    0   0   0
56    0   0   0
57    0   0   0
58    0   0   0
59    0   0   0
60    0   0   0
61   

In [271]:
e_s2 = ampl.getVariable('e').getValues().toPandas()
e_s2 = e_s2.reset_index()
e_s2.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2 = e_s2.query('valor_multiplo >0')
e_s2#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo


Identificação de cada item

In [272]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao2.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [273]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [274]:
y_s2 = ampl.getVariable('y').getValues().toPandas()
y_s2 = y_s2.reset_index()
y_s2.columns = ['Caixa','chave_loja_GCG_s2_buffer_onda', 'selecao']
y_s2 = y_s2.query('selecao>0')

df_caixas_por_loja_GCG_s2 = y_s2.pivot_table(index='Caixa', columns='chave_loja_GCG_s2_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_GCG_s2 = df_caixas_por_loja_GCG_s2.reset_index()
df_caixas_por_loja_GCG_s2.columns.name = None
df_caixas_por_loja_GCG_s2 # as colunas são as caixas

,Caixa


#### Solver Highs

In [275]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=1 timelimit=600"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 600
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 5e+01]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities   1035           50         5827
Row     residuals            0            0            0
Attempting to find feasible solution for (partial) user-supplied values of discrete variables
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 5e+01]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time    

In [276]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
36,1,13,1,1.0
218,1,73,3,1.0
586,1,196,2,1.0
925,2,84,2,1.0
944,2,90,3,1.0
...,...,...,...,...
271221,402,183,1,1.0
271260,402,196,1,5.0
271518,403,57,1,2.0
271908,403,187,1,7.0


In [277]:
df_agrupamento_loja1_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s2 = df_agrupamento_loja1_highs_s2.reset_index()
df_agrupamento_loja1_highs_s2.columns.name = None
df_agrupamento_loja1_highs_s2 # as colunas são as caixas

,item_loja,2,8,13,16,18,22,24,33,37,...,187,190,196,199,200,207,209,211,219,225
0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401,402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [278]:
df_agrupamento_loja1_highs_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s2

,item_loja,2,8,13,16,18,22,24,33,37,...,196,199,200,207,209,211,219,225,Id,chave_loja_buffer_onda
0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,10_CX_1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,10_CX_1
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,10_CX_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,10_CX_1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,10_CX_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31851,10_CX_1
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31852,10_CX_1
401,402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31853,10_CX_1
402,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31854,10_CX_1


In [279]:
df_agrupamento_loja2_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s2 = df_agrupamento_loja2_highs_s2.reset_index()
df_agrupamento_loja2_highs_s2.columns.name = None
df_agrupamento_loja2_highs_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s2

,item_loja,1,2,4,5,7,8,9,11,12,...,211,215,216,217,218,219,221,224,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34014,16_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34015,16_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [280]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,0.0
1,2,0.0,1.0,0.0
2,4,0.0,1.0,0.0
3,5,0.0,1.0,0.0
4,7,0.0,1.0,0.0
...,...,...,...,...
180,220,0.0,0.0,1.0
181,221,0.0,1.0,1.0
182,223,0.0,0.0,1.0
183,224,0.0,1.0,1.0


In [281]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,10_CX_1,16_EN_3,33_EN_3
0,1,0.0,1.0,0.0
1,2,0.0,1.0,0.0
2,4,0.0,1.0,0.0
3,5,0.0,1.0,0.0
4,7,0.0,1.0,0.0
...,...,...,...,...
180,220,0.0,0.0,1.0
181,221,0.0,1.0,1.0
182,223,0.0,0.0,1.0
183,224,0.0,1.0,1.0


In [282]:
print('Solver Highs')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[f"{col}"].sum()}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_1": 42.0
número de caixas para a chave_loja_buffer_onda "16_EN_3": 116.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 116.0


#### Solver SCIP

In [283]:
ampl.option["solver"] = "scip"
ampl.option["scip_options"] = "outlev=1 timelimit=600"
ampl.solve()

SCIP 9.0.0:   tech:outlev = 1
  lim:time = 600
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 299 upgd conss, 0 impls, 299 clqs
   (2.0s) probing: 51/67950 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (2.0s) probing aborted: 50/50 successive totally useless probings
   (2.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (9.0s) symmetry computation finished: 274 generators found (max: 274, log10 of symmetry group size: inf) (symcode time: 6.00)
dynamic symmetry handling statistics:
   orbitopal reduction:        3 components: 225x7, 225x3, 225x3
   orbital reduction:          1 components of sizes 232
   lexicographic reduction:  264 permutations with support sizes 450, 450, 624, 624, 450, 624, 450, 624, 624, 624, 624, 624, 624, 624, 450, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624, 624

In [284]:
e_s2_scip = ampl.getVariable('e').getValues().toPandas()
e_s2_scip = e_s2_scip.reset_index()
e_s2_scip.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_scip = e_s2_scip.query('valor_multiplo >0')
e_s2_scip#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
0,1,1,1,1
1,1,1,2,1
674,1,225,3,1
678,2,2,1,1
1270,2,199,2,1
...,...,...,...,...
270414,401,139,1,9
271098,402,142,1,6
271365,403,6,1,1
271731,403,128,1,8


In [285]:
df_agrupamento_loja1_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_scip_s2 = df_agrupamento_loja1_scip_s2.reset_index()
df_agrupamento_loja1_scip_s2.columns.name = None
df_agrupamento_loja1_scip_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_scip_s2

,item_loja,1,2,3,5,6,7,9,13,17,...,175,185,197,198,205,213,220,222,Id,chave_loja_buffer_onda
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,10_CX_1
1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,10_CX_1
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16,10_CX_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,10_CX_1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,10_CX_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31851,10_CX_1
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31852,10_CX_1
401,402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31853,10_CX_1
402,403,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31854,10_CX_1


In [286]:
df_agrupamento_loja2_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_scip_s2 = df_agrupamento_loja2_scip_s2.reset_index()
df_agrupamento_loja2_scip_s2.columns.name = None
df_agrupamento_loja2_scip_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_scip_s2

,item_loja,1,2,3,4,6,7,8,9,10,...,215,216,220,221,222,223,224,225,Id,chave_loja_buffer_onda
0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34013,16_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34014,16_EN_3
2,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34015,16_EN_3
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34016,16_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34017,16_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57776,16_EN_3
307,308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57777,16_EN_3
308,309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57778,16_EN_3
309,310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57779,16_EN_3


In [288]:
df_agrupamento_loja3_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_scip_s2 = df_agrupamento_loja3_scip_s2.reset_index()
df_agrupamento_loja3_scip_s2.columns.name = None
df_agrupamento_loja3_scip_s2['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_scip_s2

,item_loja,1,2,3,4,5,6,7,8,9,...,218,219,220,221,222,223,224,225,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34201,33_EN_3
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34202,33_EN_3
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,34203,33_EN_3
3,4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34204,33_EN_3
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37205,33_EN_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58412,33_EN_3
316,317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58413,33_EN_3
317,318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58414,33_EN_3
318,319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,58415,33_EN_3


In [289]:
y_s2_scip = ampl.getVariable('y').getValues().toPandas()
y_s2_scip = y_s2_scip.reset_index()
y_s2_scip.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_scip = y_s2_scip.query('selecao>0')

df_caixas_por_loja_scip_s2 = y_s2_scip.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_scip_s2 = df_caixas_por_loja_scip_s2.reset_index()
df_caixas_por_loja_scip_s2.columns.name = None
df_caixas_por_loja_scip_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,0.0,1.0,1.0
4,5,1.0,0.0,1.0
...,...,...,...,...
183,221,0.0,1.0,1.0
184,222,1.0,1.0,1.0
185,223,0.0,1.0,1.0
186,224,0.0,1.0,1.0


In [290]:
df_caixas_por_loja_scip_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_scip_s2

,caixas,10_CX_1,16_EN_3,33_EN_3
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,0.0,1.0,1.0
4,5,1.0,0.0,1.0
...,...,...,...,...
183,221,0.0,1.0,1.0
184,222,1.0,1.0,1.0
185,223,0.0,1.0,1.0
186,224,0.0,1.0,1.0


In [291]:
print('Solver SCIP')
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_scip_s2[f"{col}"].sum()}')

Solver SCIP
número de caixas para a chave_loja_buffer_onda "10_CX_1": 48.0
número de caixas para a chave_loja_buffer_onda "16_EN_3": 125.0
número de caixas para a chave_loja_buffer_onda "33_EN_3": 118.0
